# Week 10: Trajectory Optimization

### Topics Covered

- Cost Functions (smoothness, speed, safety); Model Predictive Control (MPC) for planning and control integration; Real-time constraints

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon, FancyArrowPatch
from matplotlib.animation import FuncAnimation
from scipy.optimize import minimize
from scipy.interpolate import CubicSpline

# Set random seed for reproducibility
np.random.seed(42)

## 1. Trajectory Optimization Fundamentals

Trajectory optimization finds the **best** trajectory that satisfies constraints while minimizing a cost function.

### Trajectory vs. Path

- **Path**: Geometric curve in space (no time information)
  - Example: $(x(s), y(s))$ where $s \in [0, 1]$
  
- **Trajectory**: Path with timing information
  - Example: $(x(t), y(t), v(t))$ where $t$ is time

### Optimization Problem Formulation

**Decision Variables:** Trajectory parameters $\mathbf{z} = [x_0, x_1, \ldots, x_N, u_0, u_1, \ldots, u_{N-1}]$

**Minimize:**
$$J(\mathbf{z}) = \phi(x_N) + \sum_{k=0}^{N-1} L(x_k, u_k)$$

**Subject to:**
- **Dynamics:** $x_{k+1} = f(x_k, u_k)$
- **Initial conditions:** $x_0 = x_{init}$
- **Goal constraints:** $x_N \in \mathcal{X}_{goal}$
- **State constraints:** $x_k \in \mathcal{X}_{safe}$
- **Control constraints:** $u_k \in \mathcal{U}$

Where:
- $x_k$: State at time step $k$ (position, velocity, heading)
- $u_k$: Control input (steering, acceleration)
- $\phi(x_N)$: Terminal cost
- $L(x_k, u_k)$: Stage cost

### Cost Function Components

**1. Smoothness:**
$$J_{smooth} = \int_0^T \left( \left(\frac{d^2x}{dt^2}\right)^2 + \left(\frac{d^2y}{dt^2}\right)^2 \right) dt$$
Penalizes high accelerations (jerk minimization)

**2. Speed:**
$$J_{speed} = \int_0^T (v_{ref} - v(t))^2 dt$$
Tracks desired speed

**3. Safety (Obstacle Avoidance):**
$$J_{safety} = \sum_{obstacles} \max(0, d_{safe} - d(x, obs))^2$$
Penalizes being too close to obstacles

**4. Control Effort:**
$$J_{control} = \int_0^T (u(t)^T R u(t)) dt$$
Minimizes steering and acceleration inputs

**Total Cost:**
$$J = w_1 J_{smooth} + w_2 J_{speed} + w_3 J_{safety} + w_4 J_{control}$$

---

## 2. Polynomial Trajectory Generation

Polynomial trajectories provide smooth, differentiable paths ideal for robotic motion.

### Quintic (5th Order) Polynomial

For a 1D trajectory from $(p_0, v_0, a_0)$ to $(p_f, v_f, a_f)$ over time $T$:

$$p(t) = c_0 + c_1 t + c_2 t^2 + c_3 t^3 + c_4 t^4 + c_5 t^5$$

**Boundary Conditions:**
- $p(0) = p_0, \quad \dot{p}(0) = v_0, \quad \ddot{p}(0) = a_0$
- $p(T) = p_f, \quad \dot{p}(T) = v_f, \quad \ddot{p}(T) = a_f$

This gives us 6 equations for 6 unknowns $(c_0, \ldots, c_5)$.

**Matrix Form:**
$$\begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 2 & 0 & 0 & 0 \\ 1 & T & T^2 & T^3 & T^4 & T^5 \\ 0 & 1 & 2T & 3T^2 & 4T^3 & 5T^4 \\ 0 & 0 & 2 & 6T & 12T^2 & 20T^3 \end{bmatrix} \begin{bmatrix} c_0 \\ c_1 \\ c_2 \\ c_3 \\ c_4 \\ c_5 \end{bmatrix} = \begin{bmatrix} p_0 \\ v_0 \\ a_0 \\ p_f \\ v_f \\ a_f \end{bmatrix}$$

**Advantages:**
- Smooth up to 2nd derivative (continuous acceleration)
- Exact boundary condition satisfaction
- Analytically computable jerk and snap

**Applications:**
- Robotic manipulator motion
- Drone waypoint navigation
- Lane change maneuvers

In [ ]:
# Quintic Polynomial Trajectory Generator

class QuinticPolynomial:
    """Quintic (5th order) polynomial for smooth 1D trajectory."""
    
    def __init__(self, p0, v0, a0, pf, vf, af, T):
        """
        Generate quintic polynomial trajectory.
        
        Parameters:
        - p0, v0, a0: Initial position, velocity, acceleration
        - pf, vf, af: Final position, velocity, acceleration
        - T: Duration
        """
        # Coefficient matrix
        A = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 2, 0, 0, 0],
            [1, T, T**2, T**3, T**4, T**5],
            [0, 1, 2*T, 3*T**2, 4*T**3, 5*T**4],
            [0, 0, 2, 6*T, 12*T**2, 20*T**3]
        ])
        
        # Boundary conditions
        b = np.array([p0, v0, a0, pf, vf, af])
        
        # Solve for coefficients
        self.coeffs = np.linalg.solve(A, b)
        self.T = T
    
    def position(self, t):
        """Compute position at time t."""
        c = self.coeffs
        return c[0] + c[1]*t + c[2]*t**2 + c[3]*t**3 + c[4]*t**4 + c[5]*t**5
    
    def velocity(self, t):
        """Compute velocity at time t."""
        c = self.coeffs
        return c[1] + 2*c[2]*t + 3*c[3]*t**2 + 4*c[4]*t**3 + 5*c[5]*t**4
    
    def acceleration(self, t):
        """Compute acceleration at time t."""
        c = self.coeffs
        return 2*c[2] + 6*c[3]*t + 12*c[4]*t**2 + 20*c[5]*t**3
    
    def jerk(self, t):
        """Compute jerk (derivative of acceleration) at time t."""
        c = self.coeffs
        return 6*c[3] + 24*c[4]*t + 60*c[5]*t**2


def demonstrate_quintic_trajectory():
    """Demonstrate quintic polynomial trajectory for lane change."""
    
    # Lane change scenario
    # Start: lane center at y=0, moving forward at 20 m/s
    # End: lane center at y=3.5, moving forward at 20 m/s
    # Duration: 3 seconds
    
    # Lateral motion (y-direction)
    y0, v_y0, a_y0 = 0.0, 0.0, 0.0  # Start in left lane
    yf, v_yf, a_yf = 3.5, 0.0, 0.0  # End in right lane
    T = 3.0
    
    quintic_y = QuinticPolynomial(y0, v_y0, a_y0, yf, v_yf, a_yf, T)
    
    # Longitudinal motion (x-direction) - constant velocity
    v_x = 20.0  # m/s
    
    # Generate trajectory
    t = np.linspace(0, T, 100)
    y_traj = np.array([quintic_y.position(ti) for ti in t])
    vy_traj = np.array([quintic_y.velocity(ti) for ti in t])
    ay_traj = np.array([quintic_y.acceleration(ti) for ti in t])
    jerk_traj = np.array([quintic_y.jerk(ti) for ti in t])
    
    x_traj = v_x * t  # Constant velocity in x
    
    # Visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # 1. 2D Trajectory (top view)
    ax1 = axes[0, 0]
    ax1.plot(x_traj, y_traj, 'b-', linewidth=3, label='Vehicle Path')
    ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.5, linewidth=2, label='Left Lane')
    ax1.axhline(y=3.5, color='gray', linestyle='--', alpha=0.5, linewidth=2, label='Right Lane')
    ax1.plot(x_traj[0], y_traj[0], 'go', markersize=12, label='Start')
    ax1.plot(x_traj[-1], y_traj[-1], 'ro', markersize=12, label='End')
    
    # Draw vehicle at several positions
    for i in [0, 25, 50, 75, 99]:
        vehicle_length, vehicle_width = 4.5, 2.0
        rect = Rectangle((x_traj[i] - vehicle_length/2, y_traj[i] - vehicle_width/2),
                        vehicle_length, vehicle_width, 
                        fill=False, edgecolor='blue', alpha=0.3)
        ax1.add_patch(rect)
    
    ax1.set_xlabel('Longitudinal Position X (m)', fontsize=11)
    ax1.set_ylabel('Lateral Position Y (m)', fontsize=11)
    ax1.set_title('Lane Change Trajectory (Top View)', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([-1, 5])
    
    # 2. Lateral Position over Time
    ax2 = axes[0, 1]
    ax2.plot(t, y_traj, 'b-', linewidth=2)
    ax2.axhline(y=0, color='g', linestyle='--', alpha=0.5, label='Left Lane')
    ax2.axhline(y=3.5, color='r', linestyle='--', alpha=0.5, label='Right Lane')
    ax2.set_xlabel('Time (s)', fontsize=11)
    ax2.set_ylabel('Lateral Position Y (m)', fontsize=11)
    ax2.set_title('Lateral Position Profile', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Lateral Velocity
    ax3 = axes[0, 2]
    ax3.plot(t, vy_traj, 'g-', linewidth=2)
    ax3.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax3.set_xlabel('Time (s)', fontsize=11)
    ax3.set_ylabel('Lateral Velocity (m/s)', fontsize=11)
    ax3.set_title('Lateral Velocity Profile', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # 4. Lateral Acceleration
    ax4 = axes[1, 0]
    ax4.plot(t, ay_traj, 'r-', linewidth=2)
    ax4.axhline(y=2.0, color='orange', linestyle='--', alpha=0.7, label='Comfort Limit (±2 m/s²)')
    ax4.axhline(y=-2.0, color='orange', linestyle='--', alpha=0.7)
    ax4.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax4.set_xlabel('Time (s)', fontsize=11)
    ax4.set_ylabel('Lateral Acceleration (m/s²)', fontsize=11)
    ax4.set_title('Lateral Acceleration Profile', fontsize=12, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Jerk
    ax5 = axes[1, 1]
    ax5.plot(t, jerk_traj, 'm-', linewidth=2)
    ax5.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax5.set_xlabel('Time (s)', fontsize=11)
    ax5.set_ylabel('Jerk (m/s³)', fontsize=11)
    ax5.set_title('Jerk Profile (Smoothness Measure)', fontsize=12, fontweight='bold')
    ax5.grid(True, alpha=0.3)
    
    # 6. Metrics Summary
    ax6 = axes[1, 2]
    ax6.axis('off')
    
    max_vy = np.max(np.abs(vy_traj))
    max_ay = np.max(np.abs(ay_traj))
    max_jerk = np.max(np.abs(jerk_traj))
    
    metrics_text = f"""
    QUINTIC TRAJECTORY METRICS
    {'='*40}
    
    Duration:              {T:.2f} s
    Lateral Displacement:  {yf - y0:.2f} m
    
    Max Lateral Velocity:  {max_vy:.3f} m/s
    Max Lateral Accel:     {max_ay:.3f} m/s²
    Max Jerk:              {max_jerk:.3f} m/s³
    
    Boundary Conditions:
      Start: y={y0:.1f}m, v={v_y0:.1f}m/s, a={a_y0:.1f}m/s²
      End:   y={yf:.1f}m, v={v_yf:.1f}m/s, a={a_yf:.1f}m/s²
    
    Comfort Check:
      Accel < 2.0 m/s²:    {'✓ PASS' if max_ay < 2.0 else '✗ FAIL'}
      Smooth (continuous): ✓ PASS
    
    Polynomial Coefficients:
    """
    
    for i, c in enumerate(quintic_y.coeffs):
        metrics_text += f"  c{i} = {c:10.6f}\n"
    
    ax6.text(0.1, 0.95, metrics_text, transform=ax6.transAxes,
             fontsize=10, verticalalignment='top', family='monospace',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    print("="*70)
    print("QUINTIC POLYNOMIAL TRAJECTORY GENERATION")
    print("="*70)
    print(f"Scenario: Lane change from y=0m to y=3.5m in {T}s")
    print(f"Max lateral velocity:     {max_vy:.3f} m/s")
    print(f"Max lateral acceleration: {max_ay:.3f} m/s²")
    print(f"Max jerk:                 {max_jerk:.3f} m/s³")
    print(f"Comfort limit (2 m/s²):   {'SATISFIED' if max_ay < 2.0 else 'VIOLATED'}")
    print("="*70)

demonstrate_quintic_trajectory()

In [ ]:
# Model Predictive Control with Obstacle Avoidance

class ObstacleAvoidanceMPC:
    """MPC with soft obstacle avoidance constraints."""
    
    def __init__(self, horizon=15, dt=0.1, Q=None, R=None, obstacle_weight=5000.0):
        """
        Initialize MPC with obstacle avoidance.
        
        Parameters:
        - horizon: Prediction horizon steps
        - dt: Time step
        - Q: State cost matrix (4x4)
        - R: Control cost matrix (2x2)
        - obstacle_weight: Penalty for approaching obstacles
        """
        self.N = horizon
        self.dt = dt
        
        # Cost matrices
        self.Q = Q if Q is not None else np.diag([10.0, 10.0, 1.0, 1.0])  # [x, y, vx, vy]
        self.R = R if R is not None else np.diag([0.1, 0.1])  # [ax, ay]
        self.obstacle_weight = obstacle_weight
        
        # Control limits
        self.u_max = np.array([3.0, 3.0])  # Max acceleration (m/s²)
        self.u_min = -self.u_max
        
        # Obstacle parameters
        self.obstacle_radius = 1.5  # Obstacle radius (m)
        self.safety_margin = 0.5    # Additional safety buffer (m)
    
    def dynamics(self, state, control):
        """2D point mass dynamics."""
        x, y, vx, vy = state
        ax, ay = control
        
        x_next = x + vx * self.dt
        y_next = y + vy * self.dt
        vx_next = vx + ax * self.dt
        vy_next = vy + ay * self.dt
        
        return np.array([x_next, y_next, vx_next, vy_next])
    
    def obstacle_cost(self, state, obstacles):
        """
        Compute soft constraint penalty for obstacle proximity.
        
        Uses quadratic penalty inside safety zone.
        """
        x, y = state[0], state[1]
        penalty = 0.0
        
        for obs_x, obs_y in obstacles:
            dist = np.sqrt((x - obs_x)**2 + (y - obs_y)**2)
            min_dist = self.obstacle_radius + self.safety_margin
            
            if dist < min_dist:
                # Quadratic penalty that increases as distance decreases
                penalty += (min_dist - dist)**2
        
        return self.obstacle_weight * penalty
    
    def cost(self, z, x0, x_ref, obstacles_traj):
        """
        Total cost function including tracking, control, and obstacle avoidance.
        
        z: Flattened control sequence
        x0: Initial state
        x_ref: Reference trajectory (N+1 x 4)
        obstacles_traj: List of obstacle positions at each horizon step
        """
        controls = z.reshape((self.N, 2))
        states = [x0]
        total_cost = 0.0
        
        for k in range(self.N):
            # State tracking cost
            x_error = states[k] - x_ref[k]
            total_cost += x_error @ self.Q @ x_error
            
            # Control cost
            u = controls[k]
            total_cost += u @ self.R @ u
            
            # Obstacle avoidance cost
            if k < len(obstacles_traj):
                total_cost += self.obstacle_cost(states[k], obstacles_traj[k])
            
            # Propagate dynamics
            x_next = self.dynamics(states[k], u)
            states.append(x_next)
        
        # Terminal cost (higher weight)
        x_error_final = states[-1] - x_ref[-1]
        total_cost += x_error_final @ (self.Q * 10) @ x_error_final
        
        # Terminal obstacle cost
        if len(obstacles_traj) > 0:
            total_cost += self.obstacle_cost(states[-1], obstacles_traj[-1])
        
        return total_cost
    
    def solve(self, x0, x_ref, obstacles_traj):
        """
        Solve MPC optimization problem.
        
        Returns: Optimal control sequence (N x 2)
        """
        # Initial guess (zero acceleration)
        u_init = np.zeros(self.N * 2)
        
        # Control bounds
        bounds = [(self.u_min[i % 2], self.u_max[i % 2]) for i in range(self.N * 2)]
        
        # Solve
        result = minimize(
            fun=lambda z: self.cost(z, x0, x_ref, obstacles_traj),
            x0=u_init,
            method='SLSQP',
            bounds=bounds,
            options={'maxiter': 100, 'disp': False, 'ftol': 1e-4}
        )
        
        u_opt = result.x.reshape((self.N, 2))
        return u_opt


def demonstrate_mpc_obstacle_avoidance():
    """Demonstrate MPC with dynamic obstacle avoidance."""
    
    # Reference trajectory (straight path to goal)
    start = np.array([0.0, 0.0, 0.0, 0.0])  # [x, y, vx, vy]
    goal = np.array([50.0, 0.0, 0.0, 0.0])
    
    # Moving obstacles
    obstacle1_start = np.array([15.0, -2.0])
    obstacle2_start = np.array([30.0, 2.0])
    obstacle3_start = np.array([40.0, 0.0])
    
    # Obstacle velocities
    v_obs1 = np.array([0.0, 0.5])   # Moving up
    v_obs2 = np.array([0.0, -0.3])  # Moving down
    v_obs3 = np.array([-0.2, 0.0])  # Moving left
    
    # Initialize MPC
    mpc = ObstacleAvoidanceMPC(horizon=15, dt=0.15, obstacle_weight=8000.0)
    
    # Simulation
    x_current = start.copy()
    num_steps = 80
    
    states = [x_current]
    controls = []
    obstacle_history = []
    
    print("Running MPC with obstacle avoidance...")
    
    for step in range(num_steps):
        # Update obstacle positions
        obs1_pos = obstacle1_start + v_obs1 * step * mpc.dt
        obs2_pos = obstacle2_start + v_obs2 * step * mpc.dt
        obs3_pos = obstacle3_start + v_obs3 * step * mpc.dt
        
        current_obstacles = [obs1_pos, obs2_pos, obs3_pos]
        obstacle_history.append(current_obstacles)
        
        # Generate reference trajectory (straight line to goal)
        x_ref_horizon = []
        for k in range(mpc.N + 1):
            alpha = min(1.0, (step + k) / num_steps)
            x_ref_k = start * (1 - alpha) + goal * alpha
            x_ref_horizon.append(x_ref_k)
        x_ref_horizon = np.array(x_ref_horizon)
        
        # Predict obstacle positions over horizon
        obstacles_traj = []
        for k in range(mpc.N + 1):
            future_step = step + k
            obs1_future = obstacle1_start + v_obs1 * future_step * mpc.dt
            obs2_future = obstacle2_start + v_obs2 * future_step * mpc.dt
            obs3_future = obstacle3_start + v_obs3 * future_step * mpc.dt
            obstacles_traj.append([obs1_future, obs2_future, obs3_future])
        
        # Solve MPC
        u_opt = mpc.solve(x_current, x_ref_horizon, obstacles_traj)
        
        # Apply first control
        u_apply = u_opt[0]
        controls.append(u_apply)
        
        # Simulate system
        x_current = mpc.dynamics(x_current, u_apply)
        states.append(x_current)
        
        if step % 10 == 0:
            print(f"  Step {step}/{num_steps} - Position: ({x_current[0]:.1f}, {x_current[1]:.1f})")
    
    states = np.array(states)
    controls = np.array(controls)
    
    # Visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # 1. Trajectory with obstacles
    ax1 = axes[0, 0]
    
    # Plot vehicle trajectory
    ax1.plot(states[:, 0], states[:, 1], 'b-', linewidth=3, label='Vehicle Path', zorder=5)
    ax1.plot(states[0, 0], states[0, 1], 'go', markersize=12, label='Start', zorder=6)
    ax1.plot(states[-1, 0], states[-1, 1], 'r*', markersize=15, label='End', zorder=6)
    
    # Reference path
    ax1.plot([start[0], goal[0]], [start[1], goal[1]], 'g--', alpha=0.5, linewidth=2, label='Reference')
    
    # Plot obstacle trajectories
    for i, obs_start, v_obs, color in [(0, obstacle1_start, v_obs1, 'red'),
                                        (1, obstacle2_start, v_obs2, 'orange'),
                                        (2, obstacle3_start, v_obs3, 'purple')]:
        obs_traj = [obs_start + v_obs * step * mpc.dt for step in range(num_steps)]
        obs_traj = np.array(obs_traj)
        ax1.plot(obs_traj[:, 0], obs_traj[:, 1], '--', color=color, alpha=0.3, linewidth=1)
        
        # Plot obstacle at several timesteps
        for step_idx in [0, num_steps//3, 2*num_steps//3, num_steps-1]:
            obs_pos = obs_start + v_obs * step_idx * mpc.dt
            circle = Circle(obs_pos, mpc.obstacle_radius, color=color, alpha=0.2, zorder=1)
            ax1.add_patch(circle)
            
            # Safety margin
            circle_safe = Circle(obs_pos, mpc.obstacle_radius + mpc.safety_margin, 
                               fill=False, edgecolor=color, linestyle=':', alpha=0.5, zorder=1)
            ax1.add_patch(circle_safe)
    
    ax1.set_xlabel('X Position (m)', fontsize=11)
    ax1.set_ylabel('Y Position (m)', fontsize=11)
    ax1.set_title('MPC Obstacle Avoidance Trajectory', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    ax1.axis('equal')
    ax1.set_xlim([-5, 55])
    ax1.set_ylim([-8, 8])
    
    # 2. Distance to closest obstacle over time
    ax2 = axes[0, 1]
    time_vec = np.arange(num_steps) * mpc.dt
    
    min_distances = []
    for i, obs_list in enumerate(obstacle_history):
        distances = [np.linalg.norm(states[i, :2] - obs) for obs in obs_list]
        min_distances.append(min(distances))
    
    ax2.plot(time_vec, min_distances, 'b-', linewidth=2)
    ax2.axhline(y=mpc.obstacle_radius, color='r', linestyle='--', 
               linewidth=2, label='Obstacle Radius', alpha=0.7)
    ax2.axhline(y=mpc.obstacle_radius + mpc.safety_margin, color='orange', 
               linestyle='--', linewidth=2, label='Safety Margin', alpha=0.7)
    ax2.fill_between(time_vec, 0, mpc.obstacle_radius, color='red', alpha=0.1)
    ax2.set_xlabel('Time (s)', fontsize=11)
    ax2.set_ylabel('Distance to Closest Obstacle (m)', fontsize=11)
    ax2.set_title('Safety Distance Monitoring', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 10])
    
    # 3. Y position deviation
    ax3 = axes[0, 2]
    ax3.plot(time_vec, states[:num_steps, 1], 'b-', linewidth=2)
    ax3.axhline(y=0, color='g', linestyle='--', alpha=0.5, label='Reference Y=0')
    ax3.set_xlabel('Time (s)', fontsize=11)
    ax3.set_ylabel('Y Position (m)', fontsize=11)
    ax3.set_title('Lateral Deviation for Avoidance', fontsize=12, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Control inputs
    ax4 = axes[1, 0]
    ax4.plot(time_vec, controls[:, 0], label='ax (longitudinal)', linewidth=2)
    ax4.plot(time_vec, controls[:, 1], label='ay (lateral)', linewidth=2)
    ax4.axhline(mpc.u_max[0], color='r', linestyle='--', alpha=0.5, label='Limits')
    ax4.axhline(mpc.u_min[0], color='r', linestyle='--', alpha=0.5)
    ax4.set_xlabel('Time (s)', fontsize=11)
    ax4.set_ylabel('Acceleration (m/s²)', fontsize=11)
    ax4.set_title('Control Inputs', fontsize=12, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Velocity profile
    ax5 = axes[1, 1]
    speed = np.linalg.norm(states[:, 2:4], axis=1)
    ax5.plot(time_vec, speed[:num_steps], 'b-', linewidth=2)
    ax5.set_xlabel('Time (s)', fontsize=11)
    ax5.set_ylabel('Speed (m/s)', fontsize=11)
    ax5.set_title('Velocity Profile', fontsize=12, fontweight='bold')
    ax5.grid(True, alpha=0.3)
    
    # 6. Metrics
    ax6 = axes[1, 2]
    ax6.axis('off')
    
    collision_check = all(d > mpc.obstacle_radius for d in min_distances)
    safety_check = all(d > mpc.obstacle_radius + mpc.safety_margin for d in min_distances)
    
    final_error = np.linalg.norm(states[-1, :2] - goal[:2])
    mean_control = np.mean(np.linalg.norm(controls, axis=1))
    max_lateral_dev = np.max(np.abs(states[:, 1]))
    
    metrics_text = f"""
    MPC OBSTACLE AVOIDANCE METRICS
    {'='*45}
    
    Scenario:
      Start:  ({start[0]:.1f}, {start[1]:.1f}) m
      Goal:   ({goal[0]:.1f}, {goal[1]:.1f}) m
      Duration: {time_vec[-1]:.1f} s
    
    Safety:
      Min distance to obs:    {min(min_distances):.2f} m
      Collision-free:         {'✓ YES' if collision_check else '✗ NO'}
      Safety margin kept:     {'✓ YES' if safety_check else '✗ NO'}
    
    Performance:
      Final position error:   {final_error:.2f} m
      Max lateral deviation:  {max_lateral_dev:.2f} m
      Mean control effort:    {mean_control:.2f} m/s²
      Final speed:            {speed[-1]:.2f} m/s
    
    MPC Parameters:
      Horizon (N):            {mpc.N} steps
      Time step (dt):         {mpc.dt} s
      Prediction time:        {mpc.N * mpc.dt:.1f} s
      Obstacle weight:        {mpc.obstacle_weight:.0f}
    
    Result: {'SUCCESS ✓' if collision_check and final_error < 5.0 else 'PARTIAL'}
    """
    
    ax6.text(0.05, 0.95, metrics_text, transform=ax6.transAxes,
             fontsize=9.5, verticalalignment='top', family='monospace',
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    print("="*70)
    print("MPC OBSTACLE AVOIDANCE RESULTS")
    print("="*70)
    print(f"Collision-free:         {'YES' if collision_check else 'NO'}")
    print(f"Safety margin kept:     {'YES' if safety_check else 'NO'}")
    print(f"Min obstacle distance:  {min(min_distances):.2f} m")
    print(f"Final position error:   {final_error:.2f} m")
    print(f"Max lateral deviation:  {max_lateral_dev:.2f} m")
    print("="*70)

demonstrate_mpc_obstacle_avoidance()

---

## 3. Model Predictive Control (MPC)

Model Predictive Control is a **receding horizon** optimization technique that solves an optimal control problem at each time step.

### MPC Algorithm

**At each time step $k$:**

1. **Measure** current state $x(k)$
2. **Solve** finite-horizon optimal control problem:
   $$\min_{u_k, \ldots, u_{k+N-1}} \sum_{i=k}^{k+N-1} \left[ \|x_i - x_{ref,i}\|_Q^2 + \|u_i\|_R^2 \right] + \|x_{k+N} - x_{ref,k+N}\|_P^2$$
   subject to dynamics and constraints
3. **Apply** only the first control $u_k^*$
4. **Repeat** at next time step (receding horizon)

### Key Advantages

- **Handles constraints** explicitly (actuator limits, obstacle avoidance)
- **Preview capability** using future reference trajectory
- **Online optimization** adapts to disturbances
- **Multivariable** naturally handles coupled dynamics

### MPC vs. Classical Control

| Aspect | PID | LQR | MPC |
|--------|-----|-----|-----|
| Constraints | ✗ | ✗ | ✓ |
| Preview | ✗ | ✗ | ✓ |
| Optimality | ✗ | ✓ | ✓ |
| Computation | Low | Medium | High |
| Nonlinear | ✗ | ✗ | ✓ (NMPC) |

### Applications in Autonomous Vehicles

- **Adaptive Cruise Control (ACC)**: Maintain safe distance while tracking speed
- **Lane Keeping Assist**: Track lane center with steering constraints
- **Obstacle Avoidance**: Real-time replanning around dynamic obstacles
- **Parking**: Optimize maneuvers with tight space constraints

---

## Exercises

### Exercise 1: Multi-Objective Cost Function Tuning

**Objective:** Implement and tune a trajectory optimizer with multiple competing objectives.

**Task:** Create a trajectory optimization problem that balances:
1. Minimizing travel time
2. Maximizing comfort (minimizing acceleration)
3. Maintaining safe distance from obstacles

**Instructions:**
- Define cost weights for each objective
- Implement obstacle avoidance cost using distance penalties
- Visualize the Pareto front showing trade-offs between objectives
- Compare trajectories with different weight combinations

### Exercise 2: MPC with Vehicle Dynamics

**Objective:** Extend the MPC implementation to use realistic vehicle dynamics instead of point mass.

**Task:** Implement MPC with the bicycle kinematic model:

$$
\begin{align}
\dot{x} &= v \cos(\theta) \\
\dot{y} &= v \sin(\theta) \\
\dot{\theta} &= \frac{v}{L} \tan(\delta) \\
\dot{v} &= a
\end{align}
$$

Where:
- $(x, y)$ = position
- $\theta$ = heading angle
- $v$ = velocity
- $\delta$ = steering angle (control input)
- $a$ = acceleration (control input)
- $L$ = wheelbase length

**Instructions:**
- Modify the `SimpleMPC` class to use bicycle model dynamics
- Add constraints on steering angle: $|\delta| \leq \delta_{max}$
- Add constraints on steering rate: $|\dot{\delta}| \leq \dot{\delta}_{max}$
- Test on a lane change maneuver

### Exercise 3: Receding Horizon Obstacle Avoidance

**Objective:** Implement MPC-based real-time obstacle avoidance in a dynamic environment.

**Task:** Create an MPC controller that:
1. Tracks a reference path
2. Avoids moving obstacles within the prediction horizon
3. Re-plans at each time step as obstacles move

**Instructions:**
- Add obstacle positions as time-varying constraints
- Implement collision checking within the optimization cost
- Use soft constraints (penalty method) for obstacle avoidance
- Visualize the predicted trajectory and actual path in real-time
- Compare performance with different prediction horizons (N = 5, 10, 20)

# Exercise Solutions

# Exercise 1: Multi-Objective Cost Function Tuning
# TODO: Implement multi-objective trajectory optimization
# 
# Suggested approach:
# 1. Define separate cost functions for each objective
# 2. Create weighted sum: J_total = w1*J_time + w2*J_comfort + w3*J_safety
# 3. Vary weights and collect optimal solutions
# 4. Plot Pareto front showing trade-offs
#
# Example structure:
# def multi_objective_cost(traj, weights, obstacles):
#     J_time = trajectory_duration(traj)
#     J_comfort = integrate_squared_acceleration(traj)
#     J_safety = sum([penalty(distance(traj, obs)) for obs in obstacles])
#     return weights[0]*J_time + weights[1]*J_comfort + weights[2]*J_safety


# Exercise 2: MPC with Vehicle Dynamics
# TODO: Implement bicycle model in MPC
#
# Suggested approach:
# 1. Create BicycleMPC class extending SimpleMPC
# 2. Redefine dynamics() method with bicycle kinematic model
# 3. Update state vector to [x, y, theta, v]
# 4. Update control vector to [delta, a]
# 5. Add steering constraints in bounds
#
# Example structure:
# class BicycleMPC(SimpleMPC):
#     def __init__(self, wheelbase=2.7, **kwargs):
#         super().__init__(**kwargs)
#         self.L = wheelbase
#         self.delta_max = np.deg2rad(30)  # Max steering angle
#         
#     def dynamics(self, state, control):
#         x, y, theta, v = state
#         delta, a = control
#         
#         # Bicycle model (use small-angle approximation or exact)
#         x_next = x + v * np.cos(theta) * self.dt
#         y_next = y + v * np.sin(theta) * self.dt
#         theta_next = theta + (v / self.L) * np.tan(delta) * self.dt
#         v_next = v + a * self.dt
#         
#         return np.array([x_next, y_next, theta_next, v_next])


# Exercise 3: Receding Horizon Obstacle Avoidance
# TODO: Implement MPC with dynamic obstacle avoidance
#
# Suggested approach:
# 1. Extend SimpleMPC to include obstacle cost in cost function
# 2. Predict obstacle positions over horizon
# 3. Add soft constraint penalty for collision
# 4. Re-solve at each time step
#
# Example structure:
# class ObstacleAvoidanceMPC(SimpleMPC):
#     def __init__(self, obstacle_radius=1.0, safety_margin=0.5, **kwargs):
#         super().__init__(**kwargs)
#         self.r_obs = obstacle_radius
#         self.r_safe = safety_margin
#     
#     def obstacle_cost(self, state, obstacles_t):
#         """Compute collision penalty for given state and obstacle positions"""
#         x, y = state[0], state[1]
#         penalty = 0.0
#         
#         for obs in obstacles_t:
#             dist = np.sqrt((x - obs[0])**2 + (y - obs[1])**2)
#             min_dist = self.r_obs + self.r_safe
#             
#             if dist < min_dist:
#                 # Quadratic penalty inside safety zone
#                 penalty += 1000 * (min_dist - dist)**2
#         
#         return penalty
#     
#     def cost(self, z, x0, x_ref, obstacles_trajectory):
#         """Modified cost including obstacle avoidance"""
#         controls = z.reshape((self.N, 2))
#         states = [x0]
#         total_cost = 0.0
#         
#         for k in range(self.N):
#             # Tracking cost
#             x_error = states[k] - x_ref[k]
#             total_cost += x_error @ self.Q @ x_error
#             
#             # Control cost
#             u = controls[k]
#             total_cost += u @ self.R @ u
#             
#             # Obstacle avoidance cost
#             total_cost += self.obstacle_cost(states[k], obstacles_trajectory[k])
#             
#             # Dynamics
#             x_next = self.dynamics(states[k], u)
#             states.append(x_next)
#         
#         # Terminal cost
#         x_error_final = states[-1] - x_ref[-1]
#         total_cost += x_error_final @ (self.Q * 10) @ x_error_final
#         
#         return total_cost


---

## References

### Books

1. **Betts, J. T.** (2010). *Practical Methods for Optimal Control and Estimation Using Nonlinear Programming* (2nd ed.). SIAM.
   - Comprehensive treatment of trajectory optimization methods
   - Chapter 4: Direct transcription methods
   - Chapter 7: Real-time optimization

2. **Boyd, S., & Vandenberghe, L.** (2004). *Convex Optimization*. Cambridge University Press.
   - Foundation for optimization theory
   - Available free online: https://web.stanford.edu/~boyd/cvxbook/

3. **Rawlings, J. B., Mayne, D. Q., & Diehl, M.** (2017). *Model Predictive Control: Theory, Computation, and Design* (2nd ed.). Nob Hill Publishing.
   - Definitive reference for MPC
   - Chapter 1: Introduction to MPC concepts
   - Chapter 2: MPC stability and constraint handling

4. **Lavalle, S. M.** (2006). *Planning Algorithms*. Cambridge University Press.
   - Chapter 14: Optimal motion planning
   - Free online: http://planning.cs.uiuc.edu/

### Papers

5. **Coulter, R. C.** (1992). *Implementation of the Pure Pursuit Path Tracking Algorithm*. CMU-RI-TR-92-01.
   - Classic paper on trajectory tracking
   - Foundation for many autonomous vehicle controllers

6. **Werling, M., et al.** (2010). "Optimal trajectory generation for dynamic street scenarios in a Frenet Frame." *IEEE International Conference on Robotics and Automation (ICRA)*.
   - Frenet frame approach to trajectory optimization
   - Widely used in industry for highway driving

7. **Ziegler, J., & Stiller, C.** (2009). "Spatiotemporal state lattices for fast trajectory planning in dynamic on-road driving scenarios." *IEEE/RSJ International Conference on Intelligent Robots and Systems*.
   - State lattice planning with temporal dimension
   - Used in DARPA Urban Challenge winner

8. **Paden, B., et al.** (2016). "A Survey of Motion Planning and Control Techniques for Self-Driving Urban Vehicles." *IEEE Transactions on Intelligent Vehicles*, 1(1), 33-55.
   - Excellent overview of planning and control methods
   - Section IV covers trajectory optimization

9. **Verschueren, R., et al.** (2019). "Towards a modular software package for embedded optimization." *IFAC-PapersOnLine*, 52(1), 164-169.
   - Modern computational methods for real-time MPC
   - Code generation for embedded systems

### Online Resources

10. **MathWorks - Understanding Model Predictive Control**
    - https://www.mathworks.com/videos/series/understanding-model-predictive-control.html
    - Video series explaining MPC fundamentals

11. **CasADi - Optimization Framework**
    - https://web.casadi.org/
    - Powerful symbolic framework for numerical optimization
    - Widely used for trajectory optimization in research

12. **ACADO Toolkit**
    - http://acado.github.io/
    - Open-source toolkit for automatic control and dynamic optimization
    - Implements multiple direct methods

13. **CVXPy - Convex Optimization**
    - https://www.cvxpy.org/
    - Python library for convex optimization
    - Example: https://www.cvxpy.org/examples/applications/optimal_control.html

### Software & Tools

14. **Drake** - Model-based design and verification for robotics
    - https://drake.mit.edu/
    - Trajectory optimization examples: https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/%2Ftrajectory_optimization.ipynb

15. **OMPL** - Open Motion Planning Library
    - https://ompl.kavrakilab.org/
    - Includes optimization-based planners

16. **PyDy** - Python Dynamics
    - https://www.pydy.org/
    - Symbolic dynamics for complex mechanical systems

### Research Groups & Courses

17. **Stanford AA203: Optimal and Learning-Based Control**
    - Course materials: https://stanfordasl.github.io/aa203/
    - Lecture notes on trajectory optimization and MPC

18. **MIT 6.832: Underactuated Robotics**
    - http://underactuated.mit.edu/
    - Chapter 10: Trajectory optimization
    - Interactive notebooks with examples

19. **Berkeley MPC Lab**
    - https://borrelli.me.berkeley.edu/
    - Research on MPC for autonomous vehicles
    - Publications and code repositories

### Related Topics

20. **Kelly, M.** (2017). "An Introduction to Trajectory Optimization: How to Do Your Own Direct Collocation." *SIAM Review*, 59(4), 849-904.
    - Tutorial paper with MATLAB code
    - Excellent practical introduction to direct methods

21. **Schulman, J., et al.** (2013). "Finding Locally Optimal, Collision-Free Trajectories with Sequential Convex Optimization." *Robotics: Science and Systems*.
    - TrajOpt algorithm for robot motion planning
    - Handles non-convex constraints efficiently